# init 
importing libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

# Reading_cvs

In [2]:
df = pd.read_csv('../data/athletes_sport_details.csv')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,4,False
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,18,False
2,3,Gunnar Nielsen Aaby,M,24.0,191.0,85.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,12,False
3,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,20,False
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,20,False


# Making a model just for Gymnastics athletics

In [6]:
Gymnastics_df = df[df['Sport'] == 'Gymnastics']
Gymnastics_df.head() , Gymnastics_df.describe()

(    ID                     Name Sex   Age  Height  Weight     Team  NOC  \
 32  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 33  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 34  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 35  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 36  17  Paavo Johannes Aaltonen   M  28.0   175.0    64.0  Finland  FIN   
 
           Games  Year  Season    City       Sport  \
 32  1948 Summer  1948  Summer  London  Gymnastics   
 33  1948 Summer  1948  Summer  London  Gymnastics   
 34  1948 Summer  1948  Summer  London  Gymnastics   
 35  1948 Summer  1948  Summer  London  Gymnastics   
 36  1948 Summer  1948  Summer  London  Gymnastics   
 
                                      Event   Medal  Sport_Rank  \
 32  Gymnastics Men's Individual All-Around  Bronze          14   
 33        Gymnastics Men's Team All-Around    Gold          14   
 34         Gym

we need a little sample so we use last 25% of 18k => 4K

In [7]:
Gymnastics_df_5000_sample = Gymnastics_df[Gymnastics_df['Year'] >= 2000]
Gymnastics_df_5000_sample["ID"] = range(1, len(Gymnastics_df_5000_sample) + 1)
Gymnastics_df_5000_sample.to_csv('../data/Gymnastics_sample.csv')
Gymnastics_df_5000_sample

C:\Users\aamyr\AppData\Local\Temp\ipykernel_180\2422990920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gymnastics_df_5000_sample["ID"] = range(1, len(Gymnastics_df_5000_sample) + 1)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Sport_Rank,Has_Previous_Medal
107,1,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN,14,False
108,2,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,NaN,14,False
109,3,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,NaN,14,False
110,4,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,14,False
111,5,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Rings,NaN,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205656,4868,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN,14,True
205657,4869,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN,14,True
205658,4870,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,14,True
205659,4871,Roman Volodymyrovych Zozulia,M,25.0,164.0,59.0,Ukraine,UKR,2004 Summer,2004,Summer,Athina,Gymnastics,Gymnastics Men's Rings,NaN,14,True
